In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv('Data/RefreshTokens.csv')
active_users = pd.read_csv('Data/ActiveUsers.csv')

In [2]:
df.shape, df.UserId.nunique(), active_users.shape, active_users.UserId.nunique()

((1257506, 9), 1001461, (188369, 1), 188369)

In [3]:
df.drop(['Id', 'CreatedDate','Status','ModifiedDate','DeviceId','UsageCount', 'Token'], inplace=True, axis=1)

df['LastUsedDate'] = pd.to_datetime(df['LastUsedDate'], format='%Y-%m-%d %H:%M:%S.%f%z', errors='coerce')

In [4]:
merged_df = pd.merge(active_users, df, on='UserId')
merged_df.UserId.nunique() 

188369

In [5]:
merged_df.LastUsedDate.min(), merged_df.LastUsedDate.max()

(Timestamp('2021-09-03 17:51:20.140105+0300', tz='UTC+03:00'),
 Timestamp('2023-06-22 14:06:41.012643+0300', tz='UTC+03:00'))

In [9]:
merged_df.isnull().sum()

UserId              0
LastUsedDate    38422
dtype: int64

In [10]:
merged_df.UserId.duplicated().sum() 

86898

In [ ]:
merged_df[merged_df['LastUsedDate'].isnull()]

In [11]:
null_last_used = merged_df[merged_df['LastUsedDate'].isnull()]
null_last_used_count = null_last_used['UserId'].nunique()

merged_df.dropna(subset=['LastUsedDate'], inplace=True, axis=0)

merged_data = pd.merge(null_last_used, merged_df, on='UserId')

matched_user_count = merged_data['UserId'].nunique()

print("There are a total of {} users with missing 'LastUsedDate' column.".format(null_last_used_count))
print("{} users matched.".format(matched_user_count))
print("Detected {} users with missing last login dates. These users were removed from the dataset as they did not exhibit a pattern.".format(null_last_used_count - matched_user_count))

There are a total of 34189 users with missing 'LastUsedDate' column.
5948 users matched.
Detected 28241 users with missing last login dates. These users were removed from the dataset as they did not exhibit a pattern.


In [13]:
merged_df.UserId.nunique() 

160128

In [22]:
merged_df[merged_df['UserId'] == 710607]

,UserId,LastUsedDate
261707,710607,2022-06-20 18:32:54.917247+03:00
261708,710607,2023-03-15 16:22:22.684012+03:00
261709,710607,2023-06-22 09:08:10.138517+03:00


In [14]:
merged_df.drop_duplicates(subset=['UserId'], inplace=True, keep='last')

In [15]:
merged_df[merged_df['UserId'] == 710607]

,UserId,LastUsedDate
261709,710607,2023-06-22 09:08:10.138517+03:00


In [18]:
import pandas as pd
from datetime import timedelta

def calculate_churn_rate(df, days):
    cutoff_date = df['LastUsedDate'].max() - timedelta(days=days)
    inactive_users = df[df['LastUsedDate'] < cutoff_date]['UserId'].nunique()
    total_users = df['UserId'].nunique()
    churn_rate = round((inactive_users / total_users) * 100, 2)

    active_user_idx = df[df['LastUsedDate'] >= cutoff_date]['UserId'].unique()
    inactive_user_idx = df[df['LastUsedDate'] < cutoff_date]['UserId'].unique()

    result_df = pd.DataFrame({
        'ChurnRate': [churn_rate],
        'TotalUsers': [total_users],
        'InactiveUsers': [inactive_users],
        'ActiveUsers': [total_users - inactive_users],
        'ActiveUserIdx': [active_user_idx],
        'InactiveUserIdx': [inactive_user_idx]
    })

    return result_df

result_df = calculate_churn_rate(merged_df, 60)
result_df

,ChurnRate,TotalUsers,InactiveUsers,ActiveUsers,ActiveUserIdx,InactiveUserIdx
0,66.97,160128,107234,52894,"[3638, 1013629, 3630, 535109, 1013625, 1, 1013...","[476297, 476298, 476299, 476292, 838924, 47629..."


In [19]:
import plotly.express as px

colors = ['#D01837', '#00FF00']  

fig = px.pie(values=[result_df['InactiveUsers'][0], result_df['ActiveUsers'][0]], names=['Inactive Users', 'Active Users'], title='Churn Rate by User Activity')
fig.update_traces(marker=dict(colors=colors))

fig.show()